# Modelos de Predicción

## Importación de librerías

In [ ]:
import os.path
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

## Constantes

Rutas usuales que se ocuparán en el notebook

In [ ]:
# Salvar gráficos
save_graf = False

# RUTAS
MAIN_PATH = os.path.join("..")

IMG_PATH = os.path.join(MAIN_PATH, "imagenes")
DATA_PATH = os.path.join(MAIN_PATH, "data")

WF_FOLDER_PATH = os.path.join(DATA_PATH,
                              "wildfires_us")
WF_DATA_PATH = os.path.join(WF_FOLDER_PATH,
                            "WILDFIRES_USA.csv")
WF_DATA_COLUMNS_PATH = os.path.join(WF_FOLDER_PATH,
                                    "WILDFIRES_USA_COLUMNS.csv")

# Tamaño de la imagen
my_figsize = (10, 5)

## Funciones auxiliares

Función que se ocupará para imprimir la información (número de filas y columnas) de un DataFrame.

In [ ]:
def print_cantidad(dataframe):
    """Imprime la cantidad de datos que tiene el Data Frame.
    """
    msg_cantidad = "El dataset tiene una cantidad de {} datos y {} variables."
    print(msg_cantidad.format(dataframe.shape[0], dataframe.shape[1]))
    return None

## Carga de datos

### Columnas a ocupar

Se escojen las columnas a ocupar dependiendo de la importancia que tenga. Se omiten algunas columnas tales como las que son para el ID, como el nombre que tuvo el incendio, o la columna que indica de dónde se obtuvo el incendio; pues no deberían de afectar a la predicción.

In [ ]:
# Todas las columnas
columnas = str(pd.read_csv(WF_DATA_COLUMNS_PATH).columns[0]).split(",")

# Columnas que se ocuparán en el análisis
columnas_ocupadas = columnas.copy()

# Columnas que no se ocuparán
columnas_sin_ocupar = [x for x in columnas if x not in columnas_ocupadas]

### Carga de Datos

In [ ]:
df = pd.read_csv(WF_DATA_PATH)

# Convertimos los datos que sean fechas en ese tipo de dato
df["DISC_DATE_TIME"] = pd.to_datetime(df["DISC_DATE_TIME"])
df["CONT_DATE_TIME"] = pd.to_datetime(df["CONT_DATE_TIME"])

print_cantidad(df)

df.head()

## Preparación de la data

### Convertir variables categóricas a númericas

In [ ]:
df_copy = df.copy()

df_copy = df_copy.drop("DISC_DATE_TIME", axis=1)
df_copy = df_copy.drop("CONT_DATE_TIME", axis=1)
df_copy = df_copy.drop("COUNTY", axis=1)

columnas = [
    "STATE", 
    "DISC_MONTH",
    "DISC_DAY_OF_WEEK",
    "CONT_MONTH",
    "CONT_DAY_OF_WEEK",
    "FIRE_SIZE_CLASS",
    "FIPS_NAME",
    "STAT_CAUSE_DESCR",
]

le = preprocessing.LabelEncoder()
for col in columnas:
    df_copy[col] = le.fit_transform(df_copy[col])

### Preparación de la data de entrenamiento con la de prueba

In [ ]:
from sklearn.model_selection import train_test_split

X = df_copy.drop("STAT_CAUSE_DESCR", axis=1).values
y = df.STAT_CAUSE_DESCR.values

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.3,
                                                    random_state=0
                                                   )

## Generación de modelos

Empezamos a probar la clasificación con distintos métodos

### Naïve Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
# Entrenamos
gnb = GaussianNB()
gnb.fit(X_train, y_train)

# Predecimos
y_pred_gnb = gnb.predict(X_test)

# Mostramos
print(classification_report(y_test, y_pred_gnb))

### Clasificador de k-vecinos próximos

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
# Entrenamos
knc = KNeighborsClassifier(n_neighbors=3)
knc.fit(X_train, y_train)

# Predecimos
y_pred_knc = knc.predict(X_test)

# Mostramos
print(classification_report(y_test, y_pred_knc))

### Análisis de Discriminante Lineal

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# Entrenamos
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)

# Predecimos
y_pred_lda = lda.predict(X_test)

# Mostramos
print(classification_report(y_test, y_pred_lda))

### Análisis de Discriminante Cuadrático

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
# Entrenamos
qda = QuadraticDiscriminantAnalysis()
qda.fit(X_train, y_train)

# Predecimos
y_pred_qda = qda.predict(X_test)

# Mostramos
print(classification_report(y_test, y_pred_qda))

### Árbol de Decisión

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Entrenamos
dtc = DecisionTreeClassifier()
dtc = dtc.fit(X_train, y_train)

# Predecimos
y_pred_dtc = dtc.predict(X_test)

# Mostramos
print(classification_report(y_test, y_pred_dtc))

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# Entrenamos
rfc = RandomForestClassifier(n_estimators=50)
rfc = rfc.fit(X_train, y_train)

# Predecimos
y_pred_rfc = rfc.predict(X_test)

# Mostramos
print(classification_report(y_test, y_pred_rfc))